In [2]:
import os
import pandas as pd
import pandas_datareader as dr
import numpy as np
from tqdm import tqdm
import glob

import time
import math
import warnings
from datetime import datetime,timedelta,date
#import pandas_market_calendars as mcal
import dtale
import QuantLib as ql

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

from ib_insync.contract import * 
from ib_insync import *
import utils
from fpdf import FPDF

warnings.filterwarnings('ignore')
pd.set_option('max.rows',5000)
pd.set_option('max.columns',150)

# tlg = '/Users/david/Downloads/U3913547_20200424_20210310.tlg'
tlg = '/Users/david/Downloads/trade_log.tlg'
# tlg = '/Users/david/Downloads/osap_account.tlg'
# tlg = '/Users/david/Downloads/U3913547_20200424_20210325.tlg'
# ofx_file = '/Users/david/Downloads/U3913547_20210101_20210304.qfx'

/usr/local/lib/python3.9/site-packages/dash_bootstrap_components/_table.py:5: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [2]:
#Start connection with TWS or IB Gateway open
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
try:
    #TWS Connection Live
    ib.connect('127.0.0.1', 7497, clientId=1)
    #ib.setCallback('error', onError)
    print('Connection Accepted through TWS')
except:
    #IB Gateway connection
    ib.connect('127.0.0.1', 4001, clientId=1)
    print('Connection Accepted through Gateway')
    pass

API connection failed: ConnectionRefusedError(61, "Connect call failed ('127.0.0.1', 7497)")
Make sure API port on TWS/IBG is open
API connection failed: ConnectionRefusedError(61, "Connect call failed ('127.0.0.1', 4001)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 61] Connect call failed ('127.0.0.1', 4001)

In [ ]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

def ql_to_datetime(d):
    return datetime(d.year(), d.month(), d.dayOfMonth())

def onError(reqId, errorCode, errorString, contract):
    print("ERROR", reqId, errorCode, errorString)
    if errorCode == 200 and errorString == 'No security definition has been found for the request':
        print("Bad contract")
        pass
    elif errorCode == 1102:
        print("Restarting after outage")
        main()

In [ ]:
def parse_tlg(file):
    
    #Reads in file and parses to dataframe
    log = pd.read_csv(file,sep='delimiter', header=None,engine='python')
    log = log[0].str.split('|',expand=True)
    log = log.fillna(value=np.nan)
    
    #Variables
    headers = []
    headers_index = sorted([len(log)])
    dfs = {}
    previous_index = 0
    count=0
    
    #Iterates through df rows
    for line in range(0,len(log)):
        
        #Indexes by titles
        #TODO probably a better way to do this than by > 7. Fix later...maybe
        if len(log[0].iloc[line]) > 7:
            
            #Creates header name and index 
            header = log[0].iloc[line]
            header_index = log.index.get_loc(line)
            
            
            #Stores header name and index  to list 
            headers.append(header)
            headers_index.append(header_index)
            
            #appends multiple dataframes for each category header to dict for output
        previous_index = header_index
        headers_index = sorted(headers_index)
        
        
    for header in headers:
        index1 = headers_index[count]
        index2 = headers_index[count+1]
        dfs[header] = log.iloc[index1:index2]
        
        count += 1
        
        
    #Column Transformations to output proper format
    ['symbol'] = [2].str.split(' ').str[0]
    .columns = ['option_txs','fitid','option_ticker','opt_description',
                                          'exchange','optselltype','buy_sell_type','date','time',
                                          'currency','units','100','unit_price',
                                          'total_price','commission','currate','symbol'
                                         ]
    
    ['buy_sell_type'] = ['buy_sell_type'].str.split(';').str[0]
    
    ['strike'] = ['opt_description'].str.split(' ').str[2]
    ['opt_type']  = ['opt_description'].str.split(' ').str[-1]
    ['expiration'] = ['option_ticker'].str.split(' ').str[-1].str.split('C').str[0].str.split('P').str[0]
     = .iloc[1:]
    ['date_time'] = ['date']+' '+['time']
    pd.to_datetime(['date_time'],format='%Y%m%d %H:%M:%S')
    ['date_time'] = ['date_time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S'))
    
    return dfs

In [ ]:
# tt =dr.DataReader("AAPL", "av-intraday", 
#                start='2021-01-01',
#                end='2021-04-01',
#                api_key='9SV5T3LUBDMD729S')

In [ ]:
# tt.index = tt.index.astype('datetime64[ns]')

In [ ]:
# tt[tt.index.minute ==0]

In [ ]:
# import yfinance as yf
# data = yf.download("AAPL", start="2020-01-01", end="2021-01-01",interval='1H')

In [ ]:
def position_data (ticker,opt_tx,engine='ibkr'):
    if engine == 'ibkr':
        return get_ibkr_data(ticker,opt_tx)
    elif engine == 'av':
        return
        
        

In [ ]:
def get_ibkr_data(ticker,opt_tx):
    '''
    Returns df, txs, ticker
    '''
    
    txs = opt_tx[opt_tx['symbol'] == ticker]
    txs['date'] = pd.to_datetime(txs['date'],format='%Y%m%d')


    #Convert Expiration to DT
    txs['expiration'] = pd.to_datetime(txs['expiration'],yearfirst=True)
    txs['date_time'] = txs['date_time'].apply(lambda x: hour_rounder(pd.to_datetime(x))).astype('datetime64[ns]')
    txs['units'] = txs['units'].astype('float64')

    expir_date = txs['expiration'].max()
    currency = txs['currency'].iloc[0]
    start_date = txs['date'].min()
    days = expir_date - start_date
    today = datetime.today()

    if today > expir_date:
        days = (today-start_date) + timedelta(60)


     ####### Retrieve Data from IBKR ##########   
    contract = Stock(symbol=ticker,exchange='SMART',currency=currency)
    ib.reqContractDetails(contract)

    if days.days < 365:
        dur_str = f"{days.days} D"

    elif days.days >= 365:
        dur_str = f"{math.ceil(days.days/365)} Y"


#     ib.setCallback('error', onError)
    
    bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                                    barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)

    if util.df(bars).empty:
        contract = Stock(symbol=ticker,exchange='NASDAQ',currency=currency)
        bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                                    barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)
        if util.df(bars) == None:
            contract = Stock(symbol=ticker,exchange='NYSE',currency=currency)
            bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                                        barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)
   
    df = util.df(bars)
    df = df.rename(columns={'date':'date_time'})

    txs = pd.merge(txs,df[['date_time','open','close']],on='date_time', how='left')
    df = df[df['date_time'] >=txs['date_time'].min()-timedelta(60)]
        
    return df,txs, ticker 

In [ ]:
def graph(df,mask,ticker):
    '''
    Returns Plotly figure if passed a df containing 1 tickers transactions
    '''
    # standardize column names & create buy/sell dataframes
    df.columns = df.columns.str.lower()
    buys = mask[mask['units']>0]
    sells = mask[mask['units']<0]
    
    fig = make_subplots(rows=2, cols=1,specs=[[{"secondary_y": True}],
                                              [{"secondary_y": True}]])
    
    fig.add_trace(go.Candlestick(x=df['date_time'],
               open=df['open'], 
               high=df['high'],
               low=df['low'], 
               close=df['close'],
               yaxis='y2',
               whiskerwidth=.5,
               name='OHLC',
               line=dict(width=1),
               increasing={'line': {'width': 1}},
               decreasing={'line': {'width': 1}})

             ),

    fig.add_trace(go.Scatter(
                    x=sells['date_time'],
                    y=sells['open']*1.02,
                    mode='markers',
                    name='Sell',
                    marker_symbol='triangle-down',
                    yaxis='y2',
                    marker=dict(
                         color='red',
                         size=14,
                         line=dict(
                            color='DarkSlateGrey',
                            width=1)))
                 ),
    
    fig.add_trace(go.Scatter(
                    x=buys['date_time'],
                    y=buys['close']*.98,
                    mode='markers',
                    name='Buy',
                    marker_symbol='triangle-up',
                    yaxis='y2',
                    marker=dict(
                         color='green',
                         size=14,
                         line=dict(
                            color='DarkSlateGrey',
                            width=1)))),


    fig.update_layout(xaxis=dict(tickformat='%b-%y',rangebreaks=[
                                                                 dict(bounds=["sat", "mon"]),
#                                                                  dict(values=holidays_list),
                                                                 dict(bounds=[17, 9], pattern="hour")
                                                                ]),
                      xaxis_rangeslider_visible=True,
                      height=800,width=1400,
                      hoverdistance=0,hovermode='y',title=f"{ticker} - Total PNL: ${-mask['total_price'].astype('float64').sum()}")
    

    return fig

In [ ]:
ticker = 'BABA'
graph(ticker_dict[ticker]['ohlc'],ticker_dict[ticker]['txs'],ticker).show()

KeyError: 'BABA'

In [ ]:
def holidays(dates):

    start = pd.Timestamp(dates[0],tz=None)
    end = pd.Timestamp(dates.iloc[-1],tz=None)

    calendar = ql.UnitedStates()
    holidays = calendar.holidayList(ql.Date(start.day, start.month, start.year),
                         ql.Date(end.day, end.month, end.year))
    holidays = [ql_to_datetime(x) for x in holidays]
    
    holidays_list = []

    for each_date in holidays:
        for i in range(0,24):
            ts = each_date + timedelta(hours=i)
            holidays_list.append(pd.Timestamp(np.datetime64(ts)).strftime('%Y-%m-%d %H:%M:%S'))
    return holidays_list

In [ ]:
def datatable(txs):
    '''
    Returns a plotly datatable figure when feed a txs dataframe
    '''
    
    #Set column Names
    tx_columns = txs[['date','time','opt_description','optselltype',
                      'exchange','unit_price','units','total_price','currency','currate','commission']]

    #Need to multiply by -1 to make total more intuitive since tlg files don't indicate proper bu/sell direction
    tx_columns['total_price'] = tx_columns['total_price'].astype('float64')
    tx_columns['total_price'] = tx_columns['total_price']*-1
    tx_columns['date'] = tx_columns['date'].dt.date
    tx_columns = tx_columns.sort_values(['date','time'])

    fig = go.Figure(data=[go.Table(columnwidth = [12,8,13,11,11,9,5,10,8,8,10],
                                   header=dict(values=list(tx_columns.columns)),
                                   cells=dict(values=tx_columns.T,height=50))])
    fig.update_layout(width=1200)

    return fig

In [25]:
def ticker_data_ibkr(opt_tx):
    '''
    Returns ticker dict and missing tickers list 
    '''
    ticker_dict ={}
    missing_tickers = list()

    for ticker in tqdm(opt_tx['symbol'].unique()): 
        try:
            df, txs, symbol = get_ibkr_data(ticker,opt_tx)
            ticker_dict[ticker] = dict(ohlc=df,txs=txs)

        except:
            pass
            missing_tickers.append(str(ticker))
            
    print(f"Missing tickers include {missing_tickers}")
        
    return ticker_dict, missing_tickers


In [ ]:
def ticker_data_ibkr(opt_tx):
    '''
    Returns ticker dict and missing tickers list 
    '''
    ticker_dict ={}
    missing_tickers = list()

    for ticker in tqdm(opt_tx['symbol'].unique()): 
        try:
            df, txs, symbol = get_ibkr_data(ticker,opt_tx)
            ticker_dict[ticker] = dict(ohlc=df,txs=txs)

        except:
            pass
            missing_tickers.append(str(ticker))
            
    print(f"Missing tickers include {missing_tickers}")
        
    return ticker_dict, missing_tickers

In [26]:
#Create folder for chart plots in current directory 
def save_plots(ticker_dict):
    '''
    Saves plots to current working directory
    '''

    try:
        os.mkdir('charts')
        os.mkdir('tables')
    except:
        pass
    charts_path = os.getcwd()+"/"+"charts/"
    table_path = os.getcwd()+"/"+"tables/"

    for ticker, v in tqdm(ticker_dict.items()):
        graph(ticker_dict[ticker]['ohlc'],
              ticker_dict[ticker]['txs'],
              ticker).write_image(f"charts/{ticker}.png",format='png')
        datatable(ticker_dict[ticker]['txs']).write_image(f"tables/{ticker}.png",format='png')
    

In [27]:
dfs = parse_tlg(tlg)
opt_tx = 

In [28]:
ticker_dict, missing_tickers = ticker_data(opt_tx)

  0%|                                                    | 0/20 [00:00<?, ?it/s]Error 200, reqId 4: No security definition has been found for the request, contract: Stock(symbol='BBD', exchange='SMART', currency='CAD')
Error 200, reqId 5: No security definition has been found for the request, contract: Stock(symbol='BBD', exchange='SMART', currency='CAD')
100%|███████████████████████████████████████████| 20/20 [04:01<00:00, 12.06s/it]

Missing tickers include ['BBD', 'SCR']


In [29]:
save_plots(ticker_dict)

100%|███████████████████████████████████████████| 18/18 [00:09<00:00,  1.96it/s]


In [30]:
class PDF(FPDF):
    def lines(self):
        self.set_line_width(0.0)
        self.line(5.0,5.0,205.0,5.0) # top one
        self.line(5.0,292.0,205.0,292.0) # bottom one
        self.line(5.0,5.0,5.0,292.0) # left one
        self.line(205.0,5.0,205.0,292.0) # right one
    def add_plot(self,image_path):
        pdf.image(image_path, x=20, y=8, w=170, h = 100)
        pdf.set_font("Arial", size=12)
        pdf.ln(85)  # move 85 down
        pdf.cell(160, 10, ln=1)
    def add_table(self,image_path):
        pdf.image(image_path, x=10, y=100, w=190, h = 80)
        pdf.set_font("Arial", size=12)
        pdf.ln(85)  # move 85 downs
#         pdf.cell(200, 10, ln=1)
    def notes(self):
        self.set_line_width(0.5)
        self.rect(x=10.0, y=200.0, w=190.0,h=90.0)

In [31]:
pdf = PDF(orientation='P', unit='mm', format='A4')
tickers = [ticker for ticker,v in ticker_dict.items()]

for ticker in tqdm(tickers):
    print(ticker)
    pdf.add_page()
    pdf.lines()
    pdf.notes()
    pdf.add_plot(os.getcwd()+'/'+f"charts/{ticker}.png")
    pdf.add_table(os.getcwd()+'/'+f"tables/{ticker}.png")
    
pdf.output('/Users/david/Documents/Updated_Trades_Oct7.pdf','F')

  0%|                                                    | 0/18 [00:00<?, ?it/s]

AA


  6%|██▍                                         | 1/18 [00:03<00:53,  3.17s/it]

AAPL


 11%|████▉                                       | 2/18 [00:06<00:49,  3.09s/it]

...

UBER


 94%|████████████████████████████████████████▌  | 17/18 [00:52<00:03,  3.07s/it]

VLDR


100%|███████████████████████████████████████████| 18/18 [00:55<00:00,  3.11s/it]


''

In [ ]:
pdf_w=210
pdf_h=297

class PDF(FPDF):
    def lines(self):
        self.set_line_width(0.0)
        self.line(5.0,5.0,205.0,5.0) # top one
        self.line(5.0,292.0,205.0,292.0) # bottom one
        self.line(5.0,5.0,5.0,292.0) # left one
        self.line(205.0,5.0,205.0,292.0) # right one
    def notes(self):
        self.set_line_width(0.5)
        self.rect(10.0, 190.0, 190.0,110.0)
    def add_image(self,image_path):
        pdf.image(image_path, x=10, y=8, w=150, h = 100)
        pdf.set_font("Arial", size=12)
        pdf.ln(85)  # move 85 down
        pdf.cell(200, 10, ln=1)
        
pdf = PDF(orientation='P', unit='mm', format='A4')

pdf.add_page()
# pdf.lines()
pdf.notes()
    
pdf.output('/Users/david/Documents/test2.pdf','F')

<H1> Ticker lookup
    

In [ ]:
ticker = 'AAPL'

In [ ]:

mask = opt_tx[opt_tx['symbol'] == ticker]
mask['date'] = pd.to_datetime(mask['date'],format='%Y%m%d')


#Convert Expiration to DT
mask['expiration'] = pd.to_datetime(mask['expiration'],yearfirst=True)
mask['date_time'] = mask['date_time'].apply(lambda x: hour_rounder(pd.to_datetime(x))).astype('datetime64[ns]')
mask['units'] = mask['units'].astype('float64')

expir_date = mask['expiration'].max()
currency = mask['currency'].iloc[0]
start_date = mask['date'].min()
days = expir_date - start_date
today = datetime.today()

if today > expir_date:
    days = (today-start_date) + timedelta(60)
    print('Expired')
    



# # print(expir_date,'\n',f"{currency}\n",f'{start_date}\n',days)
# print(days)


In [ ]:
# Insert symbol and currency
contract = Stock(symbol=ticker,exchange='SMART',currency=currency)
ib.reqContractDetails(contract)

if days.days < 365:
    dur_str = f"{days.days} D"
    
elif days.days >= 365:
    dur_str = f"{math.ceil(days.days/365)} Y"
    

bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                            barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)

df = util.df(bars)
df = df.rename(columns={'date':'date_time'})

mask = pd.merge(mask,df[['date_time','open','close']],on='date_time', how='left')

In [ ]:
plot_ticker(df,mask,ticker)

In [ ]:
plot_positions('AAPL',opt_tx)    

In [ ]:
import mplfinance as mpf
setup = dict(type='ohlc')
# df = df.set_index('date_time')
mpf.plot(df,style='charles',**setup,figscale=2)

<H2> OFX File Parse

In [62]:
#file = '/Users/david/Downloads/U3913547_20210101_20210224.ofx'
file = '/Users/david/Downloads/U3913547_20210101_20211008.ofx'
import pandas as pd
from ofxparse import OfxParser
import codecs
with codecs.open(file) as fileobj:
    ofx = OfxParser.parse(fileobj)

In [32]:
def ofx_security_list():
    seclist = []
    for security in ofx.security_list:
        seclist.append([security.uniqueid, security.name, security.ticker])
    
    df = pd.DataFrame(seclist,columns=['uniqueid','name','ticker'])
    df['symbol'] = df['ticker'].str.extract('([A-Z]{4}|[A-Z]{3}|[A-Z]{2}|[A-Z]{1})')

    return df

seclist = security_list()



In [73]:
txs = ofx.account.statement.transactions

for tx in txs:
    print(tx.currency)

AttributeError: 'InvestmentTransaction' object has no attribute 'currency'

In [24]:
def ofx_transactions():

        txs = ofx.account.statement.transactions

        txs_list = []
        transfer_list = []

        for tx in txs:
                
                if tx.type in ['fee','credit','int','other']:
                        pass
                        # print([tx.type,tx.amount,tx.checknum,tx.date,tx.id,tx.mcc,tx.memo,tx.payee,tx.sic,tx.user_date])

                elif tx.type =='transfer':

                        transfer_list.append([tx.type,'',tx.settleDate, tx.memo,tx.security,tx.income_type,
                                              tx.units,tx.unit_price,tx.commission,tx.fees,tx.total,tx.tferaction]) 

                elif tx.type in ['buyopt','buystock','sellopt','sellstock']:
                        txs_list.append([tx.type, tx.tradeDate, tx.settleDate, tx.memo,tx.security,
                        tx.income_type,tx.units,tx.unit_price,tx.commission,tx.fees,tx.total,tx.tferaction]) 

        txs_columns =  ['tx_type','trade_date','settle_date', 'memo','uniqueid','income_type',
                        'units','unit_price','commission','fees','total','tferaction']
          

        txs = pd.DataFrame(txs_list,columns=txs_columns)
        return txs

In [15]:
def parse_ofx():
    seclist = security_list()
    txs = transactions()
    print(txs.columns)
    print(seclist.columns)
    
    df = pd.merge(txs,seclist,on='uniqueid', how='left').drop_duplicates()
    return df
parse_ofx()

AttributeError: 'Transaction' object has no attribute 'tradeDate'

In [27]:
txs = ofx_transactions()

In [23]:
['strike'] = ['opt_description'].str.split(' ').str[2]
['opt_type']  = ['opt_description'].str.split(' ').str[-1]
['expiration'] = ['option_ticker'].str.split(' ').str[-1].str.split('C').str[0].str.split('P').str[0]

['date_time'] = ['date']+' '+['time']
pd.to_datetime(['date_time'],format='%Y%m%d %H:%M:%S')
['date_time'] = ['date_time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S'))

{'buyopt',
 'buystock',
 'cash',
 'credit',
 'fee',
 'income',
 'int',
 'other',
 'sellopt',
 'sellstock',
 'transfer'}

In [60]:
seclist['opt_description'] = seclist['name'].str.split('  ').str[-1]
seclist['exchange'] = None
seclist['opt_type'].str.split(' ').str[-1]

KeyError: 'opt_type'

In [61]:
seclist

,uniqueid,name,ticker,symbol,opt_description,exchange
0,CA80919D1033,SCR SCORE MEDIA AND GAMING INC,SCR,SCR,SCR SCORE MEDIA AND GAMING INC,None
1,CA80919D2023,SCR SCORE MEDIA AND GAMING INC,SCR.DUP,SCR,SCR SCORE MEDIA AND GAMING INC,None
2,36467W109,GME GAMESTOP CORP-CLASS A,GME,GME,GME GAMESTOP CORP-CLASS A,None
3,680665205,OLN OLIN CORP,OLN,OLN,OLN OLIN CORP,None
4,421712139,BBD 230120C00003000 BBD.B 20JAN23 3.0 C,BBD 230120C00003000,BBD,230120C00003000 BBD.B 20JAN23 3.0 C,None
...,...,...,...,...,...,...
68,472575964,XLE 210521C00053000 XLE 21MAY21 53.0 C,XLE 210521C00053000,XLE,210521C00053000 XLE 21MAY21 53.0 C,None
69,461776424,XOM 210319C00060000 XOM 19MAR21 60.0 C,XOM 210319C00060000,XOM,210319C00060000 XOM 19MAR21 60.0 C,None
70,426940700,XOM 210618C00060000 XOM 18JUN21 60.0 C,XOM 210618C00060000,XOM,210618C00060000 XOM 18JUN21 60.0 C,None
71,438775605,ZM 210319C00400000 ZM 19MAR21 400.0 C,ZM 210319C00400000,ZM,210319C00400000 ZM 19MAR21 400.0 C,None


In [59]:
txs

,tx_type,trade_date,settle_date,memo,uniqueid,income_type,units,unit_price,commission,fees,total,tferaction
0,buyopt,2021-08-09 17:28:12,None,,421712139,,25,0.2,31.25,0,-531.25,None
1,buyopt,2021-08-09 18:37:52,None,,421712139,,15,0.2,18.75,0,-318.75,None
2,buyopt,2021-06-01 17:23:17,None,,474832639,,3,5.4,1.2234,0,-1621.2234,None
3,buyopt,2021-06-01 17:22:42,None,,492234542,,3,7.05,1.2234,0,-2116.2234,None
4,buyopt,2021-04-20 16:14:32,None,,438063362,,1,5.82,1.4528,0,-583.4528,None
...,...,...,...,...,...,...,...,...,...,...,...,...
181,sellstock,2021-02-18 01:25:00,None,,CA80919D2023,,-0.5,51.86,0,0,25.93,None
182,sellstock,2021-08-05 13:54:42,None,,CA80919D2023,,-100,38.23,1,0,3822,None
183,sellstock,2021-08-05 13:54:53,None,,CA80919D2023,,-100,38.18,1,0,3817,None
184,sellstock,2021-08-05 13:54:55,None,,CA80919D2023,,-60,38.18,0.6,0,2290.2,None
